# Set references for v9 tuning and export to ONNX


In [1]:
from core import crossval_table, fit_table
from Gaugi import mkdir_p
from sklearn.model_selection import StratifiedKFold, KFold
from multiprocessing import Process
from time import sleep
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import json
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, model_from_json


from pprint import pprint
from copy import deepcopy, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...


In [2]:
output_path = 'output/fitting'
mkdir_p(output_path)

In [3]:
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "reference")

In [4]:
etbins = [15, 20, 30, 40, 50, 100, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [5]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [6]:
best_inits = pd.read_csv('output/crossval/best_inits_v9_30bins.csv',index_col=[0])
best_inits = best_inits.loc[(best_inits.model_idx==0)] # 5 neurons for all phase spaces
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

In [7]:
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
0,v9,0,0,0,0,3,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,loose,0.982843,...,232818,187639,23001,425,23282,18764,230001,4254,232818,187639
1,v9,0,0,0,0,3,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,medium,0.982843,...,232818,187639,22781,336,23282,18764,227800,3125,232818,187639
2,v9,0,0,0,0,3,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,tight,0.982843,...,232818,187639,22765,332,23282,18764,227636,3066,232818,187639
3,v9,0,0,0,0,3,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,vloose,0.982843,...,232818,187639,23016,432,23282,18764,230157,4396,232818,187639
4,v9,0,0,0,1,2,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,loose,0.982253,...,232818,187639,23000,463,23282,18764,230001,4176,232818,187639


In [8]:
basepath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
datapath = basepath+'/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(5)]

In [9]:
# calculate all pd/fa from reference file
ref_path = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins/'
ref_path += 'new_references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.ref.npz'
ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_values = [[ {} for eta in range(5)] for et in range(5)]

from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for op_name in op_names:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            _pd = refObj.getSgnPassed(op_name)/refObj.getSgnTotal(op_name)
            _fa = refObj.getBkgPassed(op_name)/refObj.getBkgTotal(op_name)
            ref_values[et_bin][eta_bin][op_name] = {'pd':_pd, 'fa':_fa, 'pd_epsilon':0.0}

In [10]:
def getPatterns( path, etbin , etabin ):

    pidname = 'el_lhmedium'
    from kepler.pandas import load_hdf
    import numpy as np
    df = load_hdf(path)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==False) ) ]
    df = df.loc[ (df['trig_L2_cl_et']/1000  >= etbin[0]) & (df['trig_L2_cl_et']/1000  < etbin[1])]
    df = df.loc[ (abs(df['trig_L2_cl_eta'])  >= etabin[0]) & (abs(df['trig_L2_cl_eta'])  < etabin[1])]

    col_names= ['trig_L2_cl_ring_%d'%i for i in range(100)]
    rings = df[col_names].values.astype(np.float32)

    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    target = df['target'].values.astype(np.int16)
    data_rings = norm1(rings)
    avgmu = df.avgmu.values
  

    n = target.shape[0]
    data_reta   = df['trig_L2_cl_reta'].astype(np.float32).to_numpy().reshape((n,1))  / 1.
    data_eratio = df['trig_L2_cl_eratio'].astype(np.float32).to_numpy().reshape((n,1))  / 1.
    data_f1     = df['trig_L2_cl_f1'].astype(np.float32).to_numpy().reshape((n,1))  / 0.6
    data_f3     = df['trig_L2_cl_f3'].astype(np.float32).to_numpy().reshape((n,1))  / 0.04
    data_weta2  = df['trig_L2_cl_weta2'].astype(np.float32).to_numpy().reshape((n,1))  / 0.02
    data_wstot  = df['trig_L2_cl_wstot'].astype(np.float32).to_numpy().reshape((n,1))  / 1.

 
    # Fix all shower shapes variables
    data_eratio[data_eratio>10.0]=0
    data_eratio[data_eratio>1.]=1.0
    data_wstot[data_wstot<-99]=0


    data_shower = np.concatenate( (data_reta,data_eratio,data_f1,data_f3,data_weta2, data_wstot), axis=1)

    return [data_rings,data_shower], target, avgmu

In [11]:
paths.append(paths[4])
ref_values.append(ref_values[4])

## Linear correction:

In [12]:
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)

In [13]:
cv_fit = fit_table( etbins, etabins, kf )

In [14]:
best_sorts.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,op_name,max_sp_val,...,pd_ref_total,fa_ref_total,pd_val_passed,fa_val_passed,pd_val_total,fa_val_total,pd_op_passed,fa_op_passed,pd_op_total,fa_op_total
12,v9,0,0,0,3,0,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,loose,0.983178,...,232818,187639,23000,405,23282,18764,230000,4114,232818,187639
13,v9,0,0,0,3,0,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,medium,0.983178,...,232818,187639,22778,315,23282,18764,227798,3075,232818,187639
14,v9,0,0,0,3,0,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,tight,0.983178,...,232818,187639,22770,312,23282,18764,227639,3016,232818,187639
15,v9,0,0,0,3,0,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,vloose,0.983178,...,232818,187639,23016,430,23282,18764,230157,4249,232818,187639
40,v9,0,1,0,0,2,/home/jodafons/public/cern_data/tunings/r1/Zee...,0,loose,0.975637,...,140971,143657,13913,563,14098,14365,139121,5189,140971,143657


**NOTE**: This step will take something like 5 min to be ready

In [15]:
best_sorts_refit = cv_fit.fill( best_sorts , getPatterns, paths, ref_values, output_path=output_path ,
                                min_avgmu=16, max_avgmu=60)

Filling... : 100%|████████████████████| 30/30 [08:47<00:00, 17.58s/it]


In [16]:
best_sorts_refit.to_csv('output/fitting/best_sorts_v9.csv')

In [17]:
for op in op_names:
    cv_fit.dump_beamer_table( best_sorts_refit.loc[best_sorts_refit.op_name == op] ,                  
                              op+' Fitting (v9)', 'fitting_v9_'+op)

2022-03-09 17:36:20,520 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v9_tight.pdf latex code...
2022-03-09 17:36:23,523 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v9_medium.pdf latex code...
2022-03-09 17:36:26,362 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v9_loose.pdf latex code...
2022-03-09 17:36:29,137 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file fitting_v9_vloose.pdf latex code...


In [18]:
!mv *.pdf output/fitting
!mv *.tex output/fitting

## Export all tunings:

In [19]:
model_name_format = 'data17_13TeV_EGAM1_probes_lhmedium_EGAM7_vetolhvloose.model_v9.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
op_capnames = ['Tight', 'Medium', 'Loose', 'VeryLoose']
for idx, op in enumerate( ['tight','medium','loose','vloose'] ):
    cv_fit.export(best_sorts_refit.loc[best_sorts_refit.op_name==op], 
                  model_name_format.format(op=op_capnames[idx]), 
                  config_name_format.format(op=op_capnames[idx]), 
                  op, 
                  to_onnx     = True,
                  remove_last = True,
                  min_avgmu   = 16,
                  max_avgmu   = 100)

2022-03-09 17:37:44,159 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerTightTriggerConfig.conf.
2022-03-09 17:38:56,555 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerMediumTriggerConfig.conf.
2022-03-09 17:40:08,579 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerLooseTriggerConfig.conf.
2022-03-09 17:41:20,528 | Py.fit_table                            INFO Export all tuning configuration to ElectronRingerVeryLooseTriggerConfig.conf.


tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -> 12
The maximum opset needed by this model is only 11.
tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 19 -

In [20]:
!mv models output/fitting
!mv *.conf output/fitting
!mv *.tex output/fitting
!mv *.pdf output/fitting

mv: cannot stat '*.tex': No such file or directory
mv: cannot stat '*.pdf': No such file or directory
